# Improving far object Detection

## working fine

In [1]:
# create and activate python virtual environment

In [2]:
#run this once 
! pip install -r requirments.txt

You should consider upgrading via the '/Users/kartiksharma/Traafic Signal Detection/venv/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model = YOLO("./best.pt") # Replace with your trained model path

def detect_shapes_and_predict(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours with hierarchy
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for i, contour in enumerate(contours):
        # Filter only outermost contours
        if hierarchy[0][i][3] != -1:  # Check if contour has a parent
            continue

        # Ignore small contours
        area = cv2.contourArea(contour)
        if area < 500:  # Adjust this threshold to suit your application
            continue

        # Approximate the contour to a polygon
        epsilon = 0.04 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Identify shapes
        x, y, w, h = cv2.boundingRect(approx)
        shape = None
        if len(approx) == 3:
            shape = "Triangle"
        elif len(approx) == 4:
            shape = "Rectangle"
        elif len(approx) > 4:
            (cx, cy), radius = cv2.minEnclosingCircle(contour)
            if radius > 10:  # Filter small circles
                shape = "Circle"

        # If a shape is identified, predict with YOLO
        if shape:
            roi = frame[y:y+h, x:x+w]  # Crop the region of interest
            if roi.size > 0:  # Ensure ROI is not empty
                results = model.predict(roi, conf=0.8, imgsz=320)
                
                # Process YOLO predictions
                for result in results[0].boxes:
                    conf = result.conf.item()  # Confidence of the prediction
                    cls = result.cls.item()  # Class index
                    label = results[0].names[int(cls)]  # Class name

                    if conf > 0.8:  # Apply confidence threshold
                        # Draw the detected shape and classification
                        cv2.putText(frame, f"{shape}: {label} ({conf:.2f})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    return frame

# Open video capture
cap = cv2.VideoCapture(0)  # 0 for default camera, replace with video path if needed

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame for shape and sign detection
    output_frame = detect_shapes_and_predict(frame)

    # Display the output
    cv2.imshow("Real-Time Shape and Sign Detection", output_frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



0: 320x128 (no detections), 238.7ms
Speed: 7.4ms preprocess, 238.7ms inference, 15.7ms postprocess per image at shape (1, 3, 320, 128)

0: 320x224 (no detections), 254.1ms
Speed: 2.6ms preprocess, 254.1ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 224)

0: 288x320 (no detections), 211.8ms
Speed: 2.2ms preprocess, 211.8ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 320)

0: 256x320 (no detections), 362.5ms
Speed: 9.7ms preprocess, 362.5ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 320)

0: 160x320 (no detections), 175.2ms
Speed: 12.7ms preprocess, 175.2ms inference, 0.6ms postprocess per image at shape (1, 3, 160, 320)

0: 160x320 (no detections), 141.3ms
Speed: 1.7ms preprocess, 141.3ms inference, 1.8ms postprocess per image at shape (1, 3, 160, 320)

0: 160x320 (no detections), 157.7ms
Speed: 1.4ms preprocess, 157.7ms inference, 0.4ms postprocess per image at shape (1, 3, 160, 320)

0: 256x320 (no detections), 177.3ms
Speed: 1.8ms pre

: 

## Multi Threading

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import threading
import time
import queue

# Load YOLO model
model = YOLO("./Model/working/gtsrb_training/weights/best.pt")  # Replace with your trained model path

# Queue to hold frames for ROI extraction and YOLO prediction
roi_queue = queue.Queue(maxsize=10)  # Queue for storing extracted ROIs
prediction_queue = queue.Queue(maxsize=10)  # Queue for storing YOLO predictions

# Lock to prevent race conditions while accessing shared resources
frame_lock = threading.Lock()
frame = None  # Shared variable to hold the current frame

def detect_shapes_and_predict(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours with hierarchy
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for i, contour in enumerate(contours):
        # Filter only outermost contours
        if hierarchy[0][i][3] != -1:  # Check if contour has a parent
            continue

        # Ignore small contours
        area = cv2.contourArea(contour)
        if area < 500:  # Adjust this threshold to suit your application
            continue

        # Approximate the contour to a polygon
        epsilon = 0.04 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Identify shapes
        x, y, w, h = cv2.boundingRect(approx)
        shape = None
        if len(approx) == 3:
            shape = "Triangle"
        elif len(approx) == 4:
            shape = "Rectangle"
        elif len(approx) > 4:
            (cx, cy), radius = cv2.minEnclosingCircle(contour)
            if radius > 10:  # Filter small circles
                shape = "Circle"

        # If a shape is identified, crop the ROI
        if shape:
            roi = frame[y:y+h, x:x+w]  # Crop the region of interest
            if roi.size > 0:  # Ensure ROI is not empty
                roi_queue.put((roi, (x, y, w, h)))  # Add the ROI and its bounding box for later processing

    return frame

def yolo_prediction_thread():
    while True:
        # Get ROI from the queue for YOLO prediction
        if not roi_queue.empty():
            roi, bbox = roi_queue.get()

            if roi.size > 0:
                # Perform YOLO prediction
                results = model.predict(roi, conf=0.8, imgsz=256)  # Reduce `imgsz` to speed up processing

                # Process YOLO predictions
                for result in results[0].boxes:
                    conf = result.conf.item()  # Confidence of the prediction
                    cls = result.cls.item()  # Class index
                    label = results[0].names[int(cls)]  # Class name

                    if conf > 0.8:  # Apply confidence threshold
                        prediction_queue.put((bbox, label, conf))  # Store prediction result and bounding box in the queue


def video_capture_thread():
    global frame
    cap = cv2.VideoCapture(0)  # 0 for default camera

    while cap.isOpened():
        ret, frame_data = cap.read()
        if not ret:
            break

        # Acquire lock to safely update shared frame
        with frame_lock:
            frame = frame_data

        # Process the frame for shape and ROI extraction
        detect_shapes_and_predict(frame)

    cap.release()

def main():
    # Start the video capture thread
    capture_thread = threading.Thread(target=video_capture_thread, daemon=True)
    capture_thread.start()

    # Start the YOLO prediction thread
    yolo_thread = threading.Thread(target=yolo_prediction_thread, daemon=True)
    yolo_thread.start()

    while True:
        with frame_lock:
            if frame is None:
                continue

            if not prediction_queue.empty():
                bbox, label, conf = prediction_queue.get()

                # Display the output with YOLO label and confidence on the original frame
                x, y, w, h = bbox
                cv2.putText(frame, f"{label}: {conf:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # Display the updated frame with all predictions
            cv2.imshow("Real-Time Shape and YOLO Detection", frame)

            # Exit on 'q' key
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()



0: 256x96 (no detections), 242.6ms
Speed: 10.2ms preprocess, 242.6ms inference, 13.6ms postprocess per image at shape (1, 3, 256, 96)

0: 256x224 (no detections), 286.0ms
Speed: 3.4ms preprocess, 286.0ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 224)

0: 256x128 (no detections), 135.1ms
Speed: 1.1ms preprocess, 135.1ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 128)

0: 256x160 (no detections), 150.9ms
Speed: 1.8ms preprocess, 150.9ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 160)

0: 256x224 (no detections), 183.5ms
Speed: 2.6ms preprocess, 183.5ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 224)

0: 256x224 (no detections), 304.9ms
Speed: 1.8ms preprocess, 304.9ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 224)

0: 256x96 (no detections), 109.4ms
Speed: 1.7ms preprocess, 109.4ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 96)

0: 128x256 (no detections), 241.2ms
Speed: 2.6ms preproc

0: 128x256 (no detections), 188.7ms
Speed: 1.9ms preprocess, 188.7ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 256)

0: 160x256 (no detections), 369.8ms
Speed: 5.4ms preprocess, 369.8ms inference, 0.4ms postprocess per image at shape (1, 3, 160, 256)

0: 96x256 (no detections), 170.0ms
Speed: 3.1ms preprocess, 170.0ms inference, 1.0ms postprocess per image at shape (1, 3, 96, 256)

0: 64x256 (no detections), 176.2ms
Speed: 2.5ms preprocess, 176.2ms inference, 0.7ms postprocess per image at shape (1, 3, 64, 256)

0: 128x256 (no detections), 190.6ms
Speed: 1.8ms preprocess, 190.6ms inference, 0.6ms postprocess per image at shape (1, 3, 128, 256)

0: 256x256 1 Speed_Limit_20, 221.3ms
Speed: 25.8ms preprocess, 221.3ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 346.7ms
Speed: 1.4ms preprocess, 346.7ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 1 Speed_Limit_20, 436.5ms
Speed: 20.1ms prepro

: 